In [1]:
%%capture
import specs
import importlib
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

prepare_config(".", "fast")
importlib.reload(specs)

import network as nt
import brlib
import validatorlib as vlib

import copy

from cadCAD.configuration import Configuration
from cadCAD.engine import ExecutionMode, ExecutionContext, Executor
import pandas as pd

In [2]:
genesis_state = brlib.get_genesis_state(4)
validators = [vlib.ASAPValidator(genesis_state, i) for i in range(4)]
brlib.skip_genesis_block(validators)

>>> recomputing attester
>>> recomputing proposer
------ update_data
updated moves 0.00023603439331054688
updated received 0.0002779960632324219
got head 0.0003681182861328125
updated proposers/attesters 0.00038909912109375
------ update_data 0.0004050731658935547
------ update_data
updated moves 0.00017690658569335938
updated received 0.0002181529998779297
got head 0.00030684471130371094
updated proposers/attesters 0.0003268718719482422
------ update_data 0.00034308433532714844


In [3]:
set_a = nt.NetworkSet(validators=list([0,1]))
set_b = nt.NetworkSet(validators=list([1,2]))
set_c = nt.NetworkSet(validators=list([2,3]))

net = nt.Network(validators = validators, sets = list([set_a, set_b, set_c]))

In [4]:
proposer_views = [validator.data.current_proposer_duties for validator in net.validators] 
proposer_views

[[False, False, False, False],
 [True, False, True, True],
 [False, True, False, False],
 [False, False, False, False]]

In [5]:
block = net.validators[2].propose({ "attestations": [], "blocks": [] })

honest validator 2 propose a block for slot 1
block contains 0 attestations


In [6]:
nt.disseminate_block(net, 2, block)

--------- disseminate_block
going to ask to check backlog 0.0002880096435546875
-------- ask_to_check_backlog
------- check_backlog
finished recording blocks 0.0027701854705810547
finished recording attestations 0.0029692649841308594
------ update_data
updated moves 5.2928924560546875e-05
updated received 0.00012683868408203125
got head 0.00039887428283691406
>>> new head
keep searching
parent is old head
got lca 0.0005047321319580078
do nothing 0.0005476474761962891
updated proposers/attesters 0.0006337165832519531
------ update_data 0.0008077621459960938
done updating data 0.0038499832153320312
------- end check_backlog 0.003878355026245117
done all the backlog 0.011375904083251953 
-------- end ask_to_check_backlog 0.011375904083251953
backlog done 0.01226806640625 
--------- end disseminate_block 0.012269020080566406


In [7]:
block_root = hash_tree_root(block.message)

try:
    net.validators[0].store.blocks[block_root]
    print("there is a block")
except KeyError: print("no block")
    
try:
    net.validators[2].store.blocks[block_root]
    print("there is a block")
except KeyError: print("no block")

no block
there is a block


In [8]:
for validator in net.validators:
    validator.forward_by(16)
print(net.validators[0].data.slot)

------ update_data
updated moves 0.00013208389282226562
updated received 0.0002040863037109375
got head 0.0004801750183105469
updated proposers/attesters 0.0005190372467041016
------ update_data 0.0005500316619873047
2


In [9]:
committee_views = [validator.data.current_attest_slot for validator in net.validators] 
committee_views

[3, 1, 2, 0]

In [10]:
knowledge = nt.knowledge_set(net, 2)
attestation = net.validators[2].attest(knowledge)

attestation 0xa42b9f0bee50f7ea32caabd982961dbc84919411c8986ac36cbbece955a6dff0 for slot 2 by validator 2 source 0 and target 0


In [11]:
print(net.validators[2].data.slot)
print(specs.get_head(net.validators[2].store))

2
0xaa458c9e45e320fe788f6fd84e60feeaad85925ff4296097576b7f92fdd2845e


In [12]:
print(attestation)

Attestation(Container)
    aggregation_bits: SpecialBitlistView = Bitlist[2048](1 bits: 1)
    data: AttestationData = AttestationData(Container)
                                slot: Slot = 2
                                index: CommitteeIndex = 0
                                beacon_block_root: Root = 0xaa458c9e45e320fe788f6fd84e60feeaad85925ff4296097576b7f92fdd2845e
                                source: Checkpoint = Checkpoint(Container)
                                                         epoch: Epoch = 0
                                                         root: Root = 0x0000000000000000000000000000000000000000000000000000000000000000
                                target: Checkpoint = Checkpoint(Container)
                                                         epoch: Epoch = 0
                                                         root: Root = 0x6106c1a5589bbb89d73023783a877d516d28bf43c1f39e3a922f41a28ba34539
    signature: BLSSignature = 0x00000000000000000000

In [13]:
for validator in net.validators:
    validator.forward_by(8)

------ update_data
updated moves 0.00010704994201660156
updated received 0.00018405914306640625
got head 0.0007719993591308594
updated proposers/attesters 0.0008180141448974609
------ update_data 0.0008509159088134766


In [14]:
nt.disseminate_attestations(net, [(2, attestation)])

--------- disseminate_attestations 1 attestations
going to ask to check backlog 0.0008127689361572266
-------- ask_to_check_backlog
------- check_backlog
finished recording blocks 5.4836273193359375e-05
finished recording attestations 0.0021097660064697266
------ update_data
updated moves 6.29425048828125e-05
updated received 0.0001480579376220703
got head 0.0005869865417480469
updated proposers/attesters 0.0006549358367919922
------ update_data 0.0007088184356689453
done updating data 0.003143787384033203
------- end check_backlog 0.003186941146850586
done all the backlog 0.006875038146972656 
-------- end ask_to_check_backlog 0.006875038146972656
backlog done 0.00793600082397461 
--------- end disseminate_attestations 0.007936954498291016


In [15]:
print(net.validators[0].store.latest_messages)

{}


In [16]:
print(net.validators[1].store.latest_messages)

{2: LatestMessage(epoch=0, root=0xaa458c9e45e320fe788f6fd84e60feeaad85925ff4296097576b7f92fdd2845e)}


In [17]:
print(net.validators[2].store.latest_messages)

{2: LatestMessage(epoch=0, root=0xaa458c9e45e320fe788f6fd84e60feeaad85925ff4296097576b7f92fdd2845e)}


In [18]:
print(net.validators[3].store.latest_messages)

{2: LatestMessage(epoch=0, root=0xaa458c9e45e320fe788f6fd84e60feeaad85925ff4296097576b7f92fdd2845e)}


In [19]:
%%capture

prepare_config(".", "medium")
importlib.reload(specs)
importlib.reload(nt)
importlib.reload(brlib)
importlib.reload(vlib)

In [20]:
num_validators = 30
genesis_state = brlib.get_genesis_state(num_validators)
validators = [vlib.ASAPValidator(genesis_state, i) for i in range(num_validators)]
brlib.skip_genesis_block(validators)

set_a = nt.NetworkSet(validators=list(range(0, int(num_validators * 2 / 3.0))))
set_b = nt.NetworkSet(validators=list(range(int(num_validators / 2.0), num_validators)))

network = nt.Network(validators = validators, sets=list([set_a, set_b]))

>>> recomputing attester
>>> recomputing proposer
------ update_data
updated moves 0.00011587142944335938
updated received 0.00015783309936523438
got head 0.0002460479736328125
updated proposers/attesters 0.0002658367156982422
------ update_data 0.0002799034118652344
------ update_data
updated moves 8.082389831542969e-05
updated received 0.00012373924255371094
got head 0.00021195411682128906
updated proposers/attesters 0.00023174285888671875
------ update_data 0.0002460479736328125


In [21]:
initial_conditions = {
    'network': network
}

block_attestation_psub = [
    {
        'policies': {
            'action': brlib.attest_policy
        },
        'variables': {
            'network': brlib.disseminate_attestations
        }
    },
    {
        'policies': {
            'action': brlib.propose_policy
        },
        'variables': {
            'network': brlib.disseminate_blocks
        }
    },
    {
        'policies': {
        },
        'variables': {
            'network': brlib.tick
        }
    },
]

In [22]:
number_slots = specs.SLOTS_PER_EPOCH + 1
steps = number_slots * specs.SECONDS_PER_SLOT * vlib.frequency

simulation_parameters = {
    'T': range(steps),
    'N': 1,
    'M': {}
}

print("will simulate", number_slots, "slots at frequency", vlib.frequency, "moves/second")
print("total", steps, "steps")

will simulate 17 slots at frequency 1 moves/second
total 204 steps


In [23]:
config = Configuration(initial_state=initial_conditions,
                       partial_state_update_blocks=block_attestation_psub,
                       sim_config=simulation_parameters
                      )

exec_mode = ExecutionMode()
exec_context = ExecutionContext(exec_mode.single_proc)
executor = Executor(exec_context, [config]) # Pass the configuration object inside an array
raw_result, tensor = executor.execute() # The `execute()` method returns a tuple; its first elements contains the raw results
df = pd.DataFrame(raw_result)


                            __________   ____ 
          ________ __ _____/ ____/   |  / __ \
         / ___/ __` / __  / /   / /| | / / / /
        / /__/ /_/ / /_/ / /___/ ___ |/ /_/ / 
        \___/\__,_/\__,_/\____/_/  |_/_____/  
        by BlockScience
        
Execution Mode: single_proc: [<cadCAD.configuration.Configuration object at 0x1248489a0>]
Configurations: [<cadCAD.configuration.Configuration object at 0x1248489a0>]
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
attest_policy time =  0.0007827281951904297
--------- disseminate_attestations 0 attestations
going to ask to check backlog 5.7220458984375e-05
-------- ask_to_check_backlog
done all the backlog 1.71661376953125e-05 
-------- end ask_to_check_backlog 1.811981201171875e-05
backlog done 0.00029015541076660156 
--------- end disseminate_attestations 0.00029015541076660156
adding 0 to network items there are now 0 attestations
disseminate_attestations time =  0.0005030632019042969
honest validator 56 propose a block for slot 1
blo

AssertionError: 